In [5]:
#| default_exp main
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Principal
> Este módulo concentra funções auxiliares específicas que filtram os dados do banco com campos e formatação de interesse para aplicações específicas como o [appAnalise](https://github.com/EricMagalhaesDelgado/appAnalise) por exemplo.

In [6]:
#| export
from pathlib import Path
import json
from typing import Union
from datetime import datetime
from tqdm.auto import tqdm

import pandas as pd
from fastcore.test import *
from rich import print
import pyodbc
from pymongo import MongoClient
from dotenv import load_dotenv

from extracao.constants import APP_ANALISE
from extracao.reading import read_base, read_aero
from extracao.format import merge_close_rows

load_dotenv()

True

In [7]:
#| export
def bump_version(
    version: str,  # String com a versão atual
    part: int = 2,  # Parte da versão que será incrementada
) -> str:  # Retorna a versão atualizada
    version = version.split(".")
    version[part] = str(int(version[part]) + 1)
    for i in range(part + 1, 3):
        version[i] = "0"
    return ".".join(version)

In [8]:
#| export
def get_modtimes(
    pasta: Union[str, Path],  # Pasta onde estão os arquivos esperados de monitoramento
) -> dict:  # Retorna o mtime de todos os arquivos pertinentes da pasta
    """
    Retorna a data de modificação dos arquivos de dados contidos na pasta
    """
    # Pasta
    pasta = Path(pasta)
    if not pasta.is_dir():
        raise FileNotFoundError(f"Pasta {pasta} não encontrada")
    # Arquivos
    for suffix in [".parquet.gzip", ".fth", ".xlsx"]:
        if not (stel := pasta / f"stel{suffix}").is_file():
            raise FileNotFoundError(f"Arquivo {stel} não encontrado")
        if not (radcom := pasta / f"radcom{suffix}").is_file():
            raise FileNotFoundError(f"Arquivo {radcom} não encontrado")
        if not (mosaico := pasta / f"mosaico{suffix}").is_file():
            raise FileNotFoundError(f"Arquivo {mosaico} não encontrado")
        break
    if not (icao := pasta / "icao.xlsx").is_file():  # ICAO
        raise FileNotFoundError(f"Arquivo {icao} não encontrado")
    if not (pmec := pasta / "aisw.xlsx").is_file():  # PMEC
        raise FileNotFoundError(f"Arquivo {pmec} não encontrado")
    if not (geo := pasta / "aisg.xlsx").is_file():  # GEO
        raise FileNotFoundError(f"Arquivo {geo} não encontrado")
    # Modificação
    mod_stel = datetime.fromtimestamp(stel.stat().st_mtime).strftime(
        "%d/%m/%Y %H:%M:%S"
    )
    mod_radcom = datetime.fromtimestamp(radcom.stat().st_mtime).strftime(
        "%d/%m/%Y %H:%M:%S"
    )
    mod_mosaico = datetime.fromtimestamp(mosaico.stat().st_mtime).strftime(
        "%d/%m/%Y %H:%M:%S"
    )
    mod_icao = pd.read_excel(icao, engine="openpyxl", sheet_name="ExtractDate").columns[
        0
    ]
    mod_aisw = pd.read_excel(pmec, engine="openpyxl", sheet_name="ExtractDate").columns[
        0
    ]
    mod_aisg = pd.read_excel(geo, engine="openpyxl", sheet_name="ExtractDate").columns[
        0
    ]
    return {
        "STEL": mod_stel,
        "SRD": mod_radcom,
        "MOSAICO": mod_mosaico,
        "ICAO": mod_icao,
        "AISW": mod_aisw,
        "AISG": mod_aisg,
    }

In [9]:
#| export
def get_db(
    path: Union[str, Path],  # Pasta onde salvar os arquivos",
    connSQL: pyodbc.Connection = None,  # Objeto de conexão do banco SQL Server
    clientMongoDB: MongoClient = None,  # Objeto de conexão do banco MongoDB
) -> pd.DataFrame:  # Retorna o DataFrame com as bases da Anatel e da Aeronáutica
    """Lê e opcionalmente atualiza as bases da Anatel, mescla as bases da Aeronáutica, salva e retorna o arquivo
    A atualização junto às bases de dados da Anatel é efetuada caso ambos objetos de banco `connSQL` e `clientMongoDB` forem válidos`
    """
    dest = Path(path)
    dest.mkdir(parents=True, exist_ok=True)
    print(":scroll:[green]Lendo as bases de dados da Anatel...")
    # if not all([connSQL, clientMongoDB]):
    #     raise ConnectionError(f"Verifique os conectores de banco de dados: {connSQL} e {clientMongoDB}")
    rd = read_base(path, connSQL, clientMongoDB)
    rd["#Estação"] = rd["Número_Estação"]
    rd.loc[rd.Multiplicidade != "1", "#Estação"] = (
        rd.loc[rd.Multiplicidade != "1", "Número_Estação"]
        + "+"
        + rd.loc[rd.Multiplicidade != "1", "Multiplicidade"]
    )
    rd["Descrição"] = (
        "["
        + rd.Fonte.fillna("NI")
        + "] "
        + rd.Status.fillna("NI")
        + ", "
        + rd.Classe.fillna("NI")
        + ", "
        + rd.Entidade.fillna("NI").str.title()
        + " ("
        + rd.Fistel.fillna("NI")
        + ", "
        + rd["#Estação"].fillna("NI")
        + "), "
        + rd.Município.fillna("NI")
        + "/"
        + rd.UF.fillna("NI")
    )

    rd.loc[rd.Coords_Valida == "0", "Descrição"] = (
        rd.loc[rd.Coords_Valida == "0", "Descrição"] + "*"
    )

    export_columns = [
        "Frequência",
        "Latitude",
        "Longitude",
        "Descrição",
        "Num_Serviço",
        "Número_Estação",
        "Classe_Emissão",
        "Largura_Emissão(kHz)",
    ]
    rd = rd.loc[:, export_columns]
    rd.columns = APP_ANALISE
    print(":airplane:[blue]Adicionando os registros da Aeronáutica.")
    aero = read_aero(path, update=False)
    rd = merge_close_rows(rd, aero)
    print(":card_file_box:[green]Salvando os arquivos...")
    versiondb = json.loads((dest.parent / "VersionFile.json").read_text())
    mod_times = get_modtimes(path)
    mod_times["ReleaseDate"] = datetime.today().strftime("%d/%m/%Y %H:%M:%S")
    for c in ["Latitude", "Longitude"]:
        rd.loc[:, c] = rd.loc[:, c].fillna(-1).astype("float32")
    rd["Frequency"] = rd["Frequency"].astype("float64")
    rd["Description"] = rd["Description"].astype("string").fillna("NI")
    rd["Service"] = rd.Service.fillna("-1").astype("int32")
    rd["Station"] = rd.Station.fillna("-1").astype("int16")
    rd.loc[rd.Station == "", "Station"] = -1
    rd.loc[rd.BW == "", "BW"] = "-1"
    rd["BW"] = rd["BW"].astype("float32").fillna(-1)
    rd["Class"] = rd.Class.fillna("NI").astype("category")
    rd = (
        rd.drop_duplicates(keep="first")
        .sort_values(by=["Frequency", "Latitude", "Longitude"])
        .reset_index(drop=True)
    )
    rd["Id"] = [f"#{i+1}" for i in rd.index]
    rd["Id"] = rd.Id.astype("string")
    rd = rd.loc[
        :,
        [
            "Id",
            "Frequency",
            "Latitude",
            "Longitude",
            "Description",
            "Service",
            "Station",
            "Class",
            "BW",
        ],
    ]
    rd.to_parquet(f"{dest}/AnatelDB.parquet.gzip", compression="gzip", index=False)
    versiondb["anateldb"]["Version"] = bump_version(versiondb["anateldb"]["Version"])
    versiondb["anateldb"].update(mod_times)
    json.dump(versiondb, (dest.parent / "VersionFile.json").open("w"))
    print("Sucesso :zap:")
    return rd

In [10]:
import os
import warnings
from extracao.updates import connect_db
warnings.filterwarnings("ignore")


In [11]:
#| hide
folder = Path.cwd().parent / 'dados'
# conn = connect_db()
# uri = os.environ['MONGO_URI']
# mongo_client = MongoClient(uri)
# mongo_client.server_info()

In [13]:
#|eval: false
from pandas_profiling import ProfileReport
base = get_db(folder)
base = pd.read_parquet(f"{Path.cwd().parent}/dados/AnatelDB.parquet.gzip")
base['Frequency'] = base['Frequency'].astype('category')
profile = ProfileReport(base, config_file='report_config.yaml')
profile.to_notebook_iframe()

📜Lendo as bases de dados da Anatel...

✈Adicionando os registros da Aeronáutica.

  0%|          | 0/830197 [00:00<?, ?it/s]